# Final Project
Wil Gaboury and Calvin Hirsch

In [118]:
import pandas as pd
import warnings
import numpy as np

In [72]:
csgo_df = pd.read_csv('./data/csgo.csv')
csgo_df = csgo_df[pd.to_numeric(csgo_df.best_of, errors='coerce').notnull()]
csgo_df

/home/calvin/projects/cmsc320-final/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date,match_id,event_id,team_1,team_2,best_of,_map,t1_start,t2_start,1_t1,...,21_winner,22_winner,23_winner,24_winner,25_winner,26_winner,27_winner,28_winner,29_winner,30_winner
0,2020-03-01,2339402,4901,G2,Natus Vincere,5,Nuke,t,ct,4350.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-01,2339402,4901,G2,Natus Vincere,5,Dust2,ct,t,3900.0,...,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,NaN
2,2020-03-01,2339402,4901,G2,Natus Vincere,5,Mirage,t,ct,4150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Dust2,t,ct,4150.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Nuke,ct,t,4200.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43229,2017-04-04,2309263,2683,FaZe,HellRaisers,1,Cache,t,ct,4250.0,...,2.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN
43230,2017-04-04,2309262,2683,Astralis,CLG,1,Train,t,ct,4250.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN
43231,2017-04-04,2309261,2683,SK,TYLOO,1,Mirage,ct,t,4350.0,...,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,NaN
43232,2017-04-04,2309260,2683,Virtus.pro,MVP Project,1,Mirage,t,ct,4150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
csgo_df['win_ratio'] = csgo_df.loc[:, '1_winner':'30_winner'].apply(lambda x: len(x[x == 1]) - len(x[x == 2]), axis=1)
csgo_df.loc[:, '1_winner':'30_winner'] = csgo_df.loc[:, '1_winner':'30_winner'].applymap(lambda x: True if x == 1 else False if x == 2 else None)

csgo_df['t1_start_ct'] = csgo_df['t1_start'].apply(lambda x: x == 'ct')
csgo_df = csgo_df.drop(['t1_start', 't2_start'], axis=1)

csgo_df = csgo_df.rename(columns={'_map':'map'})

csgo_df[['t1_start_ct', '1_winner', '30_winner', 'map', 'win_ratio']]

,t1_start_ct,1_winner,30_winner,map,win_ratio
0,False,False,None,Nuke,-12
1,True,False,None,Dust2,-3
2,False,False,None,Mirage,-14
3,False,True,None,Dust2,11
4,True,False,None,Nuke,11
...,...,...,...,...,...
43229,False,False,None,Cache,-6
43230,False,False,None,Train,5
43231,True,False,None,Mirage,4
43232,False,False,None,Mirage,12


In [147]:
maps = ["Nuke", "Dust2", "Mirage", "Train", "Cache", "Cobblestone", "Vertigo", "Inferno", "Overpass"]

classifier_df = csgo_df.copy()
classifier_df = classifier_df.drop(['event_id', 'match_id', 'date', 'best_of', 'team_1', 'team_2'], axis=1)
classifier_df = classifier_df.drop(classifier_df.loc[:, '16_winner': '30_winner'].columns, axis=1)
classifier_df = classifier_df.drop(classifier_df.loc[:, '16_t1': '30_t1'].columns, axis=1)
classifier_df = classifier_df.drop(classifier_df.loc[:, '16_t2': '30_t2'].columns, axis=1)
classifier_df = classifier_df.query("map in list(" + str(maps) + ")")
classifier_df.reset_index(inplace = True, drop = True)
label_df = classifier_df["win_ratio"]
classifier_df = classifier_df.drop(['win_ratio'], axis=1)

classifier_df

,map,1_t1,2_t1,3_t1,4_t1,5_t1,6_t1,7_t1,8_t1,9_t1,...,7_winner,8_winner,9_winner,10_winner,11_winner,12_winner,13_winner,14_winner,15_winner,t1_start_ct
0,Nuke,4350.0,1100.0,22100.0,9350.0,25750.0,10400.0,24600.0,8150.0,26700.0,...,False,False,True,False,False,False,False,False,False,False
1,Dust2,3900.0,7400.0,23250.0,28500.0,31900.0,31700.0,18950.0,30200.0,28650.0,...,False,True,True,False,False,False,False,False,False,True
2,Mirage,4150.0,14300.0,2000.0,24800.0,9000.0,23150.0,21850.0,23700.0,10450.0,...,True,False,False,False,False,False,False,False,False,False
3,Dust2,4150.0,18050.0,21000.0,25850.0,25000.0,25000.0,27250.0,26150.0,26300.0,...,True,True,True,True,True,True,True,True,True,False
4,Nuke,4200.0,10000.0,22000.0,24500.0,27550.0,29350.0,31950.0,31850.0,31750.0,...,False,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43034,Cache,4250.0,7400.0,17200.0,18500.0,26750.0,27450.0,26950.0,26350.0,26350.0,...,False,True,False,False,False,False,False,False,False,False
43035,Train,4250.0,1600.0,24250.0,17400.0,22050.0,6600.0,25650.0,25650.0,26350.0,...,True,True,True,True,True,True,False,False,False,False
43036,Mirage,4350.0,7650.0,20000.0,21500.0,29150.0,26900.0,8750.0,20700.0,5350.0,...,False,False,False,True,True,False,False,True,True,True
43037,Mirage,4150.0,11800.0,1200.0,22950.0,25750.0,26850.0,27850.0,26150.0,26650.0,...,True,True,False,True,True,True,True,True,True,False


In [148]:
for ind in range(0, len(classifier_df)):
    for map in maps:
        classifier_df.at[ind, "is" + map] = classifier_df.at[ind, "map"] == map
del classifier_df["map"]

classifier_df.head(14)

,1_t1,2_t1,3_t1,4_t1,5_t1,6_t1,7_t1,8_t1,9_t1,10_t1,...,t1_start_ct,isNuke,isDust2,isMirage,isTrain,isCache,isCobblestone,isVertigo,isInferno,isOverpass
0,4350.0,1100.0,22100.0,9350.0,25750.0,10400.0,24600.0,8150.0,26700.0,23400.0,...,False,True,False,False,False,False,False,False,False,False
1,3900.0,7400.0,23250.0,28500.0,31900.0,31700.0,18950.0,30200.0,28650.0,30350.0,...,True,False,True,False,False,False,False,False,False,False
2,4150.0,14300.0,2000.0,24800.0,9000.0,23150.0,21850.0,23700.0,10450.0,26250.0,...,False,False,False,True,False,False,False,False,False,False
3,4150.0,18050.0,21000.0,25850.0,25000.0,25000.0,27250.0,26150.0,26300.0,27850.0,...,False,False,True,False,False,False,False,False,False,False
4,4200.0,10000.0,22000.0,24500.0,27550.0,29350.0,31950.0,31850.0,31750.0,32850.0,...,True,True,False,False,False,False,False,False,False,False
5,4550.0,16250.0,21350.0,21650.0,23550.0,24050.0,25450.0,26850.0,27850.0,24900.0,...,False,False,False,False,False,False,False,False,True,False
6,4450.0,10750.0,6550.0,25950.0,19450.0,24600.0,27200.0,30300.0,26150.0,21550.0,...,True,False,True,False,False,False,False,False,False,False
7,4550.0,14600.0,2200.0,25350.0,26950.0,27450.0,26650.0,5500.0,20950.0,26650.0,...,False,False,False,False,True,False,False,False,False,False
8,4250.0,18950.0,14350.0,25300.0,3250.0,7650.0,24250.0,22950.0,8350.0,27750.0,...,True,False,True,False,False,False,False,False,False,False
9,4250.0,22450.0,21150.0,15250.0,8750.0,23100.0,13250.0,30850.0,30250.0,22250.0,...,True,False,False,False,True,False,False,False,False,False


In [251]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.l1 = nn.Linear(55, 55)
        self.l2 = nn.Linear(55, 20)
        self.l3 = nn.Linear(20, 8)
        self.l4 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = F.sigmoid(self.l1(x))
        x = F.sigmoid(self.l2(x))
        x = F.sigmoid(self.l3(x))
        x = self.l4(x)
        return x
        

In [263]:
import torch.optim as optim

c = Classifier()

criterion = nn.BCELoss()
optimizer = optim.SGD(c.parameters(), lr=0.001, momentum=0.9)

In [264]:
import torch.utils.data as data_utils

batch_size = 32

train_features = torch.tensor(classifier_df.values.astype(np.float32))
train_labels = torch.tensor(label_df.values.astype(np.float32)).unsqueeze(-1)
train_set = data_utils.TensorDataset(train_features, train_labels)

dl = data_utils.DataLoader(train_set, batch_size=batch_size, shuffle=True)
len(dl)

1345

In [265]:
print_interval = 1
batches_per_epoch = 43039.0 / batch_size

for epoch in range(50):
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(dl, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = c(inputs)
        
        #if (epoch == 1):
        #    print(outputs)
        #    print(labels)
        #    break
            
        loss = criterion(outputs.clamp(-16, 16) / 32 + 0.5, labels / 32 + 0.5)
        loss.backward()
        optimizer.step()
                
        running_acc += np.mean(np.sign(outputs.detach().cpu().numpy()) == np.sign(labels.detach().cpu().numpy()))
        running_loss += loss.item()
    if epoch % print_interval == 0:
        print("%d - loss: %.3f, %.2f%% acc" %
              (epoch + 1, running_loss / batches_per_epoch, 100 * running_acc / batches_per_epoch))
        running_loss = 0.0

print('Finished Training')

1 - loss: 0.694, 41.27% acc
2 - loss: 0.694, 41.27% acc
3 - loss: 0.693, 48.49% acc
4 - loss: 0.693, 49.99% acc
5 - loss: 0.692, 49.99% acc
6 - loss: 0.692, 49.99% acc
7 - loss: 0.692, 49.99% acc
8 - loss: 0.692, 49.99% acc
9 - loss: 0.692, 49.99% acc
10 - loss: 0.692, 49.99% acc
11 - loss: 0.692, 49.99% acc
12 - loss: 0.692, 49.99% acc
13 - loss: 0.692, 49.99% acc
14 - loss: 0.692, 49.99% acc
15 - loss: 0.692, 49.99% acc
16 - loss: 0.692, 49.99% acc
17 - loss: 0.692, 49.99% acc
18 - loss: 0.692, 49.99% acc
19 - loss: 0.692, 49.99% acc
20 - loss: 0.692, 49.99% acc
21 - loss: 0.692, 49.99% acc
22 - loss: 0.692, 49.99% acc
23 - loss: 0.692, 49.99% acc
24 - loss: 0.692, 49.99% acc
25 - loss: 0.692, 49.99% acc


KeyboardInterrupt: 